In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from glob import glob
import cv2
from tqdm import tqdm
import segmentation_models_pytorch as smp

import torch
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from efficientunet import *
from torch.optim import lr_scheduler, Adam, SGD
from torchvision.models.segmentation import deeplabv3_resnet101

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

import albumentations as A
from albumentations.pytorch import ToTensorV2
from pycocotools.coco import COCO
import pytorch_msssim

import logging
import functools
import torch._utils
import warnings 
warnings.filterwarnings('ignore')

def seed_all(seed = 42):
    
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)  # pytorch (both CPU and CUDA)
    np.random.seed(seed)  # for numpy pseudo-random generator
    #random.seed(seed)  # set fixed value for python built-in pseudo-random generator
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    
seed_all()

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
print(device)
torch.__version__

cuda


'1.7.1+cu101'

In [2]:
category_names = ["Backgroud","UNKNOWN","General trash","Paper","Paper pack","Metal","Glass","Plastic","Styrofoam","Plastic bag","Battery","Clothing"]
class_nums = len(category_names)

def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
        self.ela_aug2 = A.Compose([A.HorizontalFlip(p=0.9), 
                                  A.VerticalFlip(p=0.9), 
                                  A.RandomRotate90(p=0.9),
                                  A.OneOf([
                                    A.MotionBlur(p=1),
                                  #  A.OpticalDistortion(p=1),
                                  #  A.GaussNoise(p=1),
                                    ], p=1),
                                   A.ElasticTransform(p=0.9),
                                   A.GridDistortion(p=0.9),
                                   A.OpticalDistortion(p=0.9),
                                   ToTensorV2()])
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        #CLAHE Convert
        #images = clahe_cvt(images)
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        #images /= 255.0
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # Unknown = 1, General trash = 2, ... , Cigarette = 11
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)
            masks = masks.astype(np.float32)

            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
                
                if self.mode == 'train' :
                    ela2_img = self.ela_aug2(image = images, mask = masks)
                    transformed = A.Compose([ToTensorV2()])(image = images, mask = masks)
                    images = transformed["image"]
                    masks = transformed["mask"]
                
                    images = (images, ela2_img["image"])#, loc_img["image"])#ela1_img["image"], ela2_img["image"])
                    masks = (masks, ela2_img["mask"])#, loc_img["mask"])#ela1_img["mask"], ela2_img["mask"])
                
            return images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                #hor_image = self.test_hor(image = images)["image"]
                #ver_image = self.test_ver(image = images)["image"]
                
                transformed = self.transform(image=images)
                images = transformed["image"]
                
                #images = (images, hor_image, ver_image)
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())
    
# train.json / validation.json / test.json 디렉토리 설정
dataset_path = 'input/data'
train_path = dataset_path + '/train.json'
val_path = dataset_path + '/val.json'
test_path = dataset_path + '/test.json'

# collate_fn needs for batch
def collate_fn(batch): return tuple(zip(*batch))

test_transform = A.Compose([
                           #A.CLAHE(),
                           #A.Resize(256, 256),
                           A.Normalize(
                                mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0
                           ),
                           ToTensorV2(),
                           ])


# test dataset
test_dataset = CustomDataLoader(data_dir=test_path, mode='test', transform=test_transform)


# DataLoader
batch_size = 16

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          num_workers=4,
                                          collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [2]:


ALIGN_CORNERS = True
BN_MOMENTUM = 0.1


if torch.__version__.startswith('0'):
    from .sync_bn.inplace_abn.bn import InPlaceABNSync
    BatchNorm2d = functools.partial(InPlaceABNSync, activation='none')
    BatchNorm2d_class = InPlaceABNSync
    relu_inplace = False
else:
    BatchNorm2d_class = BatchNorm2d = torch.nn.SyncBatchNorm
    relu_inplace = True



logger = logging.getLogger(__name__)

class ModuleHelper:

    @staticmethod
    def BNReLU(num_features, bn_type=None, **kwargs):
        return nn.Sequential(
            nn.BatchNorm2d(num_features, **kwargs),
            nn.ReLU()
        )

    @staticmethod
    def BatchNorm2d(*args, **kwargs):
        return BatchNorm2d


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class SpatialGather_Module(nn.Module):
    """
        Aggregate the context features according to the initial 
        predicted probability distribution.
        Employ the soft-weighted method to aggregate the context.
    """
    def __init__(self, cls_num=0, scale=1):
        super(SpatialGather_Module, self).__init__()
        self.cls_num = cls_num
        self.scale = scale

    def forward(self, feats, probs):
        batch_size, c, h, w = probs.size(0), probs.size(1), probs.size(2), probs.size(3)
        probs = probs.view(batch_size, c, -1)
        feats = feats.view(batch_size, feats.size(1), -1)
        feats = feats.permute(0, 2, 1) # batch x hw x c 
        probs = F.softmax(self.scale * probs, dim=2)# batch x k x hw
        ocr_context = torch.matmul(probs, feats)\
        .permute(0, 2, 1).unsqueeze(3)# batch x k x c
        return ocr_context


class _ObjectAttentionBlock(nn.Module):
    '''
    The basic implementation for object context block
    Input:
        N X C X H X W
    Parameters:
        in_channels       : the dimension of the input feature map
        key_channels      : the dimension after the key/query transform
        scale             : choose the scale to downsample the input feature maps (save memory cost)
        bn_type           : specify the bn type
    Return:
        N X C X H X W
    '''
    def __init__(self, 
                 in_channels, 
                 key_channels, 
                 scale=1, 
                 bn_type=None):
        super(_ObjectAttentionBlock, self).__init__()
        self.scale = scale
        self.in_channels = in_channels
        self.key_channels = key_channels
        self.pool = nn.MaxPool2d(kernel_size=(scale, scale))
        self.f_pixel = nn.Sequential(
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.key_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.key_channels, bn_type=bn_type),
            nn.Conv2d(in_channels=self.key_channels, out_channels=self.key_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.key_channels, bn_type=bn_type),
        )
        self.f_object = nn.Sequential(
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.key_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.key_channels, bn_type=bn_type),
            nn.Conv2d(in_channels=self.key_channels, out_channels=self.key_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.key_channels, bn_type=bn_type),
        )
        self.f_down = nn.Sequential(
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.key_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.key_channels, bn_type=bn_type),
        )
        self.f_up = nn.Sequential(
            nn.Conv2d(in_channels=self.key_channels, out_channels=self.in_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.in_channels, bn_type=bn_type),
        )

    def forward(self, x, proxy):
        batch_size, h, w = x.size(0), x.size(2), x.size(3)
        if self.scale > 1:
            x = self.pool(x)

        query = self.f_pixel(x).view(batch_size, self.key_channels, -1)
        query = query.permute(0, 2, 1)
        key = self.f_object(proxy).view(batch_size, self.key_channels, -1)
        value = self.f_down(proxy).view(batch_size, self.key_channels, -1)
        value = value.permute(0, 2, 1)

        sim_map = torch.matmul(query, key)
        sim_map = (self.key_channels**-.5) * sim_map
        sim_map = F.softmax(sim_map, dim=-1)   

        # add bg context ...
        context = torch.matmul(sim_map, value)
        context = context.permute(0, 2, 1).contiguous()
        context = context.view(batch_size, self.key_channels, *x.size()[2:])
        context = self.f_up(context)
        if self.scale > 1:
            context = F.interpolate(input=context, size=(h, w), mode='bilinear', align_corners=ALIGN_CORNERS)

        return context


class ObjectAttentionBlock2D(_ObjectAttentionBlock):
    def __init__(self, 
                 in_channels, 
                 key_channels, 
                 scale=1, 
                 bn_type=None):
        super(ObjectAttentionBlock2D, self).__init__(in_channels,
                                                     key_channels,
                                                     scale, 
                                                     bn_type=bn_type)


class SpatialOCR_Module(nn.Module):
    """
    Implementation of the OCR module:
    We aggregate the global object representation to update the representation for each pixel.
    """
    def __init__(self, 
                 in_channels, 
                 key_channels, 
                 out_channels, 
                 scale=1, 
                 dropout=0.1, 
                 bn_type=None):
        super(SpatialOCR_Module, self).__init__()
        self.object_context_block = ObjectAttentionBlock2D(in_channels, 
                                                           key_channels, 
                                                           scale, 
                                                           bn_type)
        _in_channels = 2 * in_channels

        self.conv_bn_dropout = nn.Sequential(
            nn.Conv2d(_in_channels, out_channels, kernel_size=1, padding=0, bias=False),
            ModuleHelper.BNReLU(out_channels, bn_type=bn_type),
            nn.Dropout2d(dropout)
        )

    def forward(self, feats, proxy_feats):
        context = self.object_context_block(feats, proxy_feats)

        output = self.conv_bn_dropout(torch.cat([context, feats], 1))

        return output


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=relu_inplace)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1,
                               bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion,
                               momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=relu_inplace)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        out = self.relu(out)

        return out


class HighResolutionModule(nn.Module):
    def __init__(self, num_branches, blocks, num_blocks, num_inchannels,
                 num_channels, fuse_method, multi_scale_output=True):
        super(HighResolutionModule, self).__init__()
        self._check_branches(
            num_branches, blocks, num_blocks, num_inchannels, num_channels)

        self.num_inchannels = num_inchannels
        self.fuse_method = fuse_method
        self.num_branches = num_branches

        self.multi_scale_output = multi_scale_output

        self.branches = self._make_branches(
            num_branches, blocks, num_blocks, num_channels)
        self.fuse_layers = self._make_fuse_layers()
        self.relu = nn.ReLU(inplace=relu_inplace)

    def _check_branches(self, num_branches, blocks, num_blocks,
                        num_inchannels, num_channels):
        if num_branches != len(num_blocks):
            error_msg = 'NUM_BRANCHES({}) <> NUM_BLOCKS({})'.format(
                num_branches, len(num_blocks))
            logger.error(error_msg)
            raise ValueError(error_msg)

        if num_branches != len(num_channels):
            error_msg = 'NUM_BRANCHES({}) <> NUM_CHANNELS({})'.format(
                num_branches, len(num_channels))
            logger.error(error_msg)
            raise ValueError(error_msg)

        if num_branches != len(num_inchannels):
            error_msg = 'NUM_BRANCHES({}) <> NUM_INCHANNELS({})'.format(
                num_branches, len(num_inchannels))
            logger.error(error_msg)
            raise ValueError(error_msg)

    def _make_one_branch(self, branch_index, block, num_blocks, num_channels,
                         stride=1):
        downsample = None
        if stride != 1 or \
           self.num_inchannels[branch_index] != num_channels[branch_index] * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.num_inchannels[branch_index],
                          num_channels[branch_index] * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(num_channels[branch_index] * block.expansion,
                            momentum=BN_MOMENTUM),
            )

        layers = []
        layers.append(block(self.num_inchannels[branch_index],
                            num_channels[branch_index], stride, downsample))
        self.num_inchannels[branch_index] = \
            num_channels[branch_index] * block.expansion
        for i in range(1, num_blocks[branch_index]):
            layers.append(block(self.num_inchannels[branch_index],
                                num_channels[branch_index]))

        return nn.Sequential(*layers)

    def _make_branches(self, num_branches, block, num_blocks, num_channels):
        branches = []

        for i in range(num_branches):
            branches.append(
                self._make_one_branch(i, block, num_blocks, num_channels))

        return nn.ModuleList(branches)

    def _make_fuse_layers(self):
        if self.num_branches == 1:
            return None

        num_branches = self.num_branches
        num_inchannels = self.num_inchannels
        fuse_layers = []
        for i in range(num_branches if self.multi_scale_output else 1):
            fuse_layer = []
            for j in range(num_branches):
                if j > i:
                    fuse_layer.append(nn.Sequential(
                        nn.Conv2d(num_inchannels[j],
                                  num_inchannels[i],
                                  1,
                                  1,
                                  0,
                                  bias=False),
                        nn.BatchNorm2d(num_inchannels[i], momentum=BN_MOMENTUM)))
                elif j == i:
                    fuse_layer.append(None)
                else:
                    conv3x3s = []
                    for k in range(i-j):
                        if k == i - j - 1:
                            num_outchannels_conv3x3 = num_inchannels[i]
                            conv3x3s.append(nn.Sequential(
                                nn.Conv2d(num_inchannels[j],
                                          num_outchannels_conv3x3,
                                          3, 2, 1, bias=False),
                                nn.BatchNorm2d(num_outchannels_conv3x3,
                                            momentum=BN_MOMENTUM)))
                        else:
                            num_outchannels_conv3x3 = num_inchannels[j]
                            conv3x3s.append(nn.Sequential(
                                nn.Conv2d(num_inchannels[j],
                                          num_outchannels_conv3x3,
                                          3, 2, 1, bias=False),
                                nn.BatchNorm2d(num_outchannels_conv3x3,
                                            momentum=BN_MOMENTUM),
                                nn.ReLU(inplace=relu_inplace)))
                    fuse_layer.append(nn.Sequential(*conv3x3s))
            fuse_layers.append(nn.ModuleList(fuse_layer))

        return nn.ModuleList(fuse_layers)

    def get_num_inchannels(self):
        return self.num_inchannels

    def forward(self, x):
        if self.num_branches == 1:
            return [self.branches[0](x[0])]

        for i in range(self.num_branches):
            x[i] = self.branches[i](x[i])

        x_fuse = []
        for i in range(len(self.fuse_layers)):
            y = x[0] if i == 0 else self.fuse_layers[i][0](x[0])
            for j in range(1, self.num_branches):
                if i == j:
                    y = y + x[j]
                elif j > i:
                    width_output = x[i].shape[-1]
                    height_output = x[i].shape[-2]
                    y = y + F.interpolate(
                        self.fuse_layers[i][j](x[j]),
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=ALIGN_CORNERS)
                else:
                    y = y + self.fuse_layers[i][j](x[j])
            x_fuse.append(self.relu(y))

        return x_fuse


blocks_dict = {
    'BASIC': BasicBlock,
    'BOTTLENECK': Bottleneck
}


class HighResolutionNet(nn.Module):

    def __init__(self, config, **kwargs):
        global ALIGN_CORNERS
        super(HighResolutionNet, self).__init__()
        extra = config['MODEL']['EXTRA']
        ALIGN_CORNERS = config['MODEL']['ALIGN_CORNERS']

        # stem net
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64, momentum=BN_MOMENTUM)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(64, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=relu_inplace)

        self.stage1_cfg = extra['STAGE1']
        num_channels = self.stage1_cfg['NUM_CHANNELS'][0]
        block = blocks_dict[self.stage1_cfg['BLOCK']]
        num_blocks = self.stage1_cfg['NUM_BLOCKS'][0]
        self.layer1 = self._make_layer(block, 64, num_channels, num_blocks)
        stage1_out_channel = block.expansion*num_channels

        self.stage2_cfg = extra['STAGE2']
        num_channels = self.stage2_cfg['NUM_CHANNELS']
        block = blocks_dict[self.stage2_cfg['BLOCK']]
        num_channels = [
            num_channels[i] * block.expansion for i in range(len(num_channels))]
        self.transition1 = self._make_transition_layer(
            [stage1_out_channel], num_channels)
        self.stage2, pre_stage_channels = self._make_stage(
            self.stage2_cfg, num_channels)

        self.stage3_cfg = extra['STAGE3']
        num_channels = self.stage3_cfg['NUM_CHANNELS']
        block = blocks_dict[self.stage3_cfg['BLOCK']]
        num_channels = [
            num_channels[i] * block.expansion for i in range(len(num_channels))]
        self.transition2 = self._make_transition_layer(
            pre_stage_channels, num_channels)
        self.stage3, pre_stage_channels = self._make_stage(
            self.stage3_cfg, num_channels)

        self.stage4_cfg = extra['STAGE4']
        num_channels = self.stage4_cfg['NUM_CHANNELS']
        block = blocks_dict[self.stage4_cfg['BLOCK']]
        num_channels = [
            num_channels[i] * block.expansion for i in range(len(num_channels))]
        self.transition3 = self._make_transition_layer(
            pre_stage_channels, num_channels)
        self.stage4, pre_stage_channels = self._make_stage(
            self.stage4_cfg, num_channels, multi_scale_output=True)

        last_inp_channels = np.int(np.sum(pre_stage_channels))
        ocr_mid_channels = config['MODEL']['OCR']['MID_CHANNELS']
        ocr_key_channels = config['MODEL']['OCR']['KEY_CHANNELS']

        self.conv3x3_ocr = nn.Sequential(
            nn.Conv2d(last_inp_channels, ocr_mid_channels,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(ocr_mid_channels),
            nn.ReLU(inplace=relu_inplace),
        )
        self.ocr_gather_head = SpatialGather_Module(config['DATASET']['NUM_CLASSES'])

        self.ocr_distri_head = SpatialOCR_Module(in_channels=ocr_mid_channels,
                                                 key_channels=ocr_key_channels,
                                                 out_channels=ocr_mid_channels,
                                                 scale=1,
                                                 dropout=0.05,
                                                 )
        self.cls_head = nn.Conv2d(
            ocr_mid_channels, config['DATASET']['NUM_CLASSES'], kernel_size=1, stride=1, padding=0, bias=True)

        self.aux_head = nn.Sequential(
            nn.Conv2d(last_inp_channels, last_inp_channels,
                      kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(last_inp_channels),
            nn.ReLU(inplace=relu_inplace),
            nn.Conv2d(last_inp_channels, config['DATASET']['NUM_CLASSES'],
                      kernel_size=1, stride=1, padding=0, bias=True)
        )
        
    def _make_transition_layer(
            self, num_channels_pre_layer, num_channels_cur_layer):
        num_branches_cur = len(num_channels_cur_layer)
        num_branches_pre = len(num_channels_pre_layer)

        transition_layers = []
        for i in range(num_branches_cur):
            if i < num_branches_pre:
                if num_channels_cur_layer[i] != num_channels_pre_layer[i]:
                    transition_layers.append(nn.Sequential(
                        nn.Conv2d(num_channels_pre_layer[i],
                                  num_channels_cur_layer[i],
                                  3,
                                  1,
                                  1,
                                  bias=False),
                        nn.BatchNorm2d(
                            num_channels_cur_layer[i], momentum=BN_MOMENTUM),
                        nn.ReLU(inplace=relu_inplace)))
                else:
                    transition_layers.append(None)
            else:
                conv3x3s = []
                for j in range(i+1-num_branches_pre):
                    inchannels = num_channels_pre_layer[-1]
                    outchannels = num_channels_cur_layer[i] \
                        if j == i-num_branches_pre else inchannels
                    conv3x3s.append(nn.Sequential(
                        nn.Conv2d(
                            inchannels, outchannels, 3, 2, 1, bias=False),
                        nn.BatchNorm2d(outchannels, momentum=BN_MOMENTUM),
                        nn.ReLU(inplace=relu_inplace)))
                transition_layers.append(nn.Sequential(*conv3x3s))

        return nn.ModuleList(transition_layers)

    def _make_layer(self, block, inplanes, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, momentum=BN_MOMENTUM),
            )

        layers = []
        layers.append(block(inplanes, planes, stride, downsample))
        inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(inplanes, planes))

        return nn.Sequential(*layers)

    def _make_stage(self, layer_config, num_inchannels,
                    multi_scale_output=True):
        num_modules = layer_config['NUM_MODULES']
        num_branches = layer_config['NUM_BRANCHES']
        num_blocks = layer_config['NUM_BLOCKS']
        num_channels = layer_config['NUM_CHANNELS']
        block = blocks_dict[layer_config['BLOCK']]
        fuse_method = layer_config['FUSE_METHOD']

        modules = []
        for i in range(num_modules):
            # multi_scale_output is only used last module
            if not multi_scale_output and i == num_modules - 1:
                reset_multi_scale_output = False
            else:
                reset_multi_scale_output = True
            modules.append(
                HighResolutionModule(num_branches,
                                     block,
                                     num_blocks,
                                     num_inchannels,
                                     num_channels,
                                     fuse_method,
                                     reset_multi_scale_output)
            )
            num_inchannels = modules[-1].get_num_inchannels()

        return nn.Sequential(*modules), num_inchannels

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.layer1(x)

        x_list = []
        for i in range(self.stage2_cfg['NUM_BRANCHES']):
            if self.transition1[i] is not None:
                x_list.append(self.transition1[i](x))
            else:
                x_list.append(x)
        y_list = self.stage2(x_list)

        x_list = []
        for i in range(self.stage3_cfg['NUM_BRANCHES']):
            if self.transition2[i] is not None:
                if i < self.stage2_cfg['NUM_BRANCHES']:
                    x_list.append(self.transition2[i](y_list[i]))
                else:
                    x_list.append(self.transition2[i](y_list[-1]))
            else:
                x_list.append(y_list[i])
        y_list = self.stage3(x_list)

        x_list = []
        for i in range(self.stage4_cfg['NUM_BRANCHES']):
            if self.transition3[i] is not None:
                if i < self.stage3_cfg['NUM_BRANCHES']:
                    x_list.append(self.transition3[i](y_list[i]))
                else:
                    x_list.append(self.transition3[i](y_list[-1]))
            else:
                x_list.append(y_list[i])
        x = self.stage4(x_list)

        # Upsampling
        x0_h, x0_w = x[0].size(2), x[0].size(3)
        x1 = F.interpolate(x[1], size=(x0_h, x0_w),
                        mode='bilinear', align_corners=ALIGN_CORNERS)
        x2 = F.interpolate(x[2], size=(x0_h, x0_w),
                        mode='bilinear', align_corners=ALIGN_CORNERS)
        x3 = F.interpolate(x[3], size=(x0_h, x0_w),
                        mode='bilinear', align_corners=ALIGN_CORNERS)

        feats = torch.cat([x[0], x1, x2, x3], 1)

        out_aux_seg = []

        # ocr
        out_aux = self.aux_head(feats)
        # compute contrast feature
        feats = self.conv3x3_ocr(feats)

        context = self.ocr_gather_head(feats, out_aux)
        feats = self.ocr_distri_head(feats, context)

        out = self.cls_head(feats)

        out_aux_seg.append(out_aux)
        out_aux_seg.append(out)

        return out_aux_seg

    def init_weights(self, pretrained='',):
        logger.info('=> init weights from normal distribution')
        for name, m in self.named_modules():
            if any(part in name for part in {'cls', 'aux', 'ocr'}):
                # print('skipped', name)
                continue
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.001)
            elif isinstance(m, BatchNorm2d_class):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        if os.path.isfile(pretrained):
            pretrained_dict = torch.load(pretrained, map_location={'cuda:0': 'cpu'})
            logger.info('=> loading pretrained model {}'.format(pretrained))
            model_dict = self.state_dict()
            pretrained_dict = {k.replace('last_layer', 'aux_head').replace('model.', ''): v for k, v in pretrained_dict.items()}            
            pretrained_dict = {k: v for k, v in pretrained_dict.items()
                               if k in model_dict.keys()}
            # for k, _ in pretrained_dict.items():
                # logger.info(
                #     '=> loading {} pretrained model {}'.format(k, pretrained))
            model_dict.update(pretrained_dict)
            self.load_state_dict(model_dict)
        elif pretrained:
            raise RuntimeError('No such file {}'.format(pretrained))


def get_seg_model(cfg, **kwargs):
    model = HighResolutionNet(cfg, **kwargs)
    #model.init_weights(cfg['MODEL']['PRETRAINED'])

    return model


class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )



class CrossEntropy(nn.Module):
    def __init__(self, config, ignore_label=-1, weight=None):
        super(CrossEntropy, self).__init__()
        self.ignore_label = ignore_label
        self.criterion = nn.CrossEntropyLoss(
            weight=weight,
            ignore_index=ignore_label
        )
        self.config = config
        self.focal_loss = FocalLoss()

    def _forward(self, score, target):
        ph, pw = score.size(2), score.size(3)
        h, w = target.size(1), target.size(2)
        if ph != h or pw != w:
            score = F.interpolate(input=score, size=(
                h, w), mode='bilinear', align_corners=self.config['MODEL']['ALIGN_CORNERS'])

        cross_loss = self.criterion(score, target)
        focal_loss = self.focal_loss(score, target)
        loss = 0.9*cross_loss+0.1*focal_loss
        return loss

    def forward(self, score, target):

        if self.config['MODEL']['NUM_OUTPUTS'] == 1:
            score = [score]

        weights = self.config['LOSS']['BALANCE_WEIGHTS']
        assert len(weights) == len(score)

        return sum([w * self._forward(x, target) for (w, x) in zip(weights, score)])

    

import yaml

def model_loss_define(yaml_path, train=True):
    with open(yaml_path) as f:
        cfg = yaml.load(f)

    model = get_seg_model(cfg)
    creterion = CrossEntropy(cfg)
    return model, creterion



#### SoftVote

1) 전체 모델에 대해

    1-1) 최고 모델 + 0.1 / 최저 모델 - 0.1
 
    1-2) 최고 모델 0.0, 그 다음부터 -0.05씩
 
    1-3) 상위 3개 모델에만 0.4
 
    1-4) 최고 모델에만 0.5
 
    1-5) 가중치 없이
 
 
2) 상위 3개

     2-1) 최고 모델 + 0.1 / 최저 모델 - 0.1
 
     2-2) 최고 모델 0.0, 그 다음부터 -0.05씩
 
     2-3) 최고 모델에만 0.5
 
     2-4) 가중치 없이
 
 
3) 최고 점수 기준 -0.01

     3-1) 최고 모델 + 0.1 / 최저 모델 - 0.1
     
     3-2) 최고 모델 0.0, 그 다음부터 -0.05씩
 
     3-3) 최고 모델에만 0.5
 
     3-4) 가중치 없이
 

In [11]:
#SoftVote
def multi_model_ensemble_test (models, weights, dataloader, device, hrn=9999999) :
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    for model in models :
        model.eval()
    
    file_name_list = []
    #file_list_
    preds_array = np.empty((0, size*size), dtype=np.long)
    
    with torch.no_grad():
        for step, (imgs, image_infos) in tqdm(enumerate(test_loader)) :
            x = torch.stack(imgs).to(device)

            # inference (512 x 512)
            outs = models[0](x) * weights[0]
            isFirst = True
            for model_step, (model, weight) in enumerate(zip(models, weights)) :
                if isFirst :
                    isFirst = False
                else :
                    if model_step == hrn :
                        predict = model(x)[1]
                        predict = F.interpolate(input = predict, size = (512, 512), mode = 'bilinear', align_corners = True)
                        outs += predict * weight
                    else :
                        outs += model(x) * weight
            
            if outs.shape[0] == 1 : oms = torch.argmax(outs.squeeze(), dim = 0).detach().cpu().numpy()
            else : oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
            
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)

            oms = np.array(temp_mask)
            
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            
            file_name_list.append([i['file_name'] for i in image_infos])
            
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    return file_names, preds_array



In [12]:
#---------------------1-1

models = list()
weights = list()
dir_name = "./Camper_models/"
for i, model_name in enumerate(["dv3p_16batch_adamp_512_cos.pt",\
                                "dv3p_16batch_adamp_512.pt",\
                                "dv3p_16batch_adamp_512_full.pt",\
                                "dv3p_16batch_adamp_512_red.pt",\
                                "Final9.pt",\
                                "Final4.pt",\
                                "effib4_elatrans.pt",\
                                "effib4_cross_label_focal.pt",\
                                "DLV3P_res152_5989.pt",\
                                "DLV3P_res152_5959.pt",\
                                "dv3p_16batch_adamp_512_wsj.pt"
                  ]) :
    model_name = dir_name + model_name
    if i < 4 :
        model = smp.DeepLabV3Plus(
                    encoder_name='resnext101_32x4d',
                    encoder_weights = 'ssl',
                    classes = class_nums).to(device)
        model.load_state_dict(torch.load(model_name))
    elif i < 6 :
        model = smp.DeepLabV3Plus(
                encoder_name='timm-regnety_032',
                encoder_weights = 'imagenet',
                classes = class_nums).to(device)
        model.load_state_dict(torch.load(model_name)['net'])
    elif i < 8 :
        model = smp.DeepLabV3Plus(encoder_name="timm-efficientnet-b4",
                          encoder_weights="noisy-student",
                          classes=class_nums).to(device)
        model.load_state_dict(torch.load(model_name))
    elif i < 10 :
        model = smp.DeepLabV3Plus(
                    encoder_name='resnet152',
                    encoder_weights = 'imagenet',
                    classes = class_nums).to(device)
        model.load_state_dict(torch.load(model_name))
    else :
        model, _ = model_loss_define('/opt/ml/code/seg_hrnet.yaml')
        model.to(device)
        model.load_state_dict(torch.load(model_name))
        
    models.append(model)
    weights.append(1)

#가중치 추가
weights[0] += 0.1
weights[len(models) - 1] -= 0.1
    
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device, hrn = 10)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_1_1"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')

#-------------1-2

weights[0] = 1
weights[len(models) - 1] = 1
for i, weight in enumerate(weights) :
    weights[i] = weight - (0.05 * i)
    if weights[i] < 0 : weights[i] = 0

# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device, hrn = 10)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_1_2"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')


#-------------1-3

for i, weight in enumerate(weights) :
    weights[i] = 1

weights[0] += 0.4
weights[1] += 0.4
weights[2] += 0.4
    
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device, hrn = 10)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_1_3"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')


#-------------1-4

for i, weight in enumerate(weights) :
    weights[i] = 1

weights[0] += 0.5
    
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device, hrn = 10)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_1_4"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')

#-------------1-5

for i, weight in enumerate(weights) :
    weights[i] = 1
    
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device, hrn = 10)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_1_5"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')

Start prediction.


53it [20:37, 23.34s/it]


End prediction.
soft_vote_1_1 is saved!
Start prediction.


53it [20:48, 23.56s/it]


End prediction.
soft_vote_1_2 is saved!
Start prediction.


...

End prediction.
soft_vote_1_3 is saved!
Start prediction.


53it [20:45, 23.51s/it]


End prediction.
soft_vote_1_4 is saved!
Start prediction.


53it [20:41, 23.43s/it]


End prediction.
soft_vote_1_5 is saved!


In [ ]:
#---------------------2-1

models = list()
weights = list()
dir_name = "./Camper_models/"
for model_name in ["dv3p_16batch_adamp_512_cos.pt",\
                  "dv3p_16batch_adamp_512.pt",\
                  "dv3p_16batch_adamp_512_full.pt"
                  ] :
    model_name = dir_name + model_name
    model = smp.DeepLabV3Plus(
                encoder_name='resnext101_32x4d',
                encoder_weights = 'ssl',
                classes = class_nums).to(device)
    model.load_state_dict(torch.load(model_name))
    models.append(model)
    weights.append(1)

#가중치 추가
weights[0] += 0.1
weights[len(models) - 1] -= 0.1
    
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_2_1"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')

#---------------------2-2

weights[0] = 1
weights[len(models) - 1] = 1
for i, weight in enumerate(weights) :
    weights[i] = weight - (0.05 * i)

# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_2_2"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')


#-------------2-3

for i, weight in enumerate(weights) :
    weights[i] = 1

weights[0] += 0.5
    
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_2_3"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')

#-------------2-4

for i, weight in enumerate(weights) :
    weights[i] = 1
    
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_2_4"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')

In [ ]:
#---------------------3-1

models = list()
weights = list()
dir_name = "./Camper_models/"
for model_name in ["dv3p_16batch_adamp_512_cos.pt",\
                  "dv3p_16batch_adamp_512.pt",\
                  "dv3p_16batch_adamp_512_full.pt",\
                   "dv3p_16batch_adamp_512_red.pt"
                  ] :
    model_name = dir_name + model_name
    model = smp.DeepLabV3Plus(
                encoder_name='resnext101_32x4d',
                encoder_weights = 'ssl',
                classes = class_nums).to(device)
    model.load_state_dict(torch.load(model_name))
    models.append(model)
    weights.append(1)

#가중치 추가
weights[0] += 0.1
weights[len(models) - 1] -= 0.1
    
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_3_1"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')

#---------------------3-2

weights[0] = 1
weights[len(models) - 1] = 1
for i, weight in enumerate(weights) :
    weights[i] = weight - (0.05 * i)
    if weights[i] < 0 : weights[i] = 0

# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_3_2"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')


#-------------3-3

for i, weight in enumerate(weights) :
    weights[i] = 1

weights[0] += 0.5
    
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_3_3"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')

#-------------3-4

for i, weight in enumerate(weights) :
    weights[i] = 1
    
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
# test set에 대한 prediction
file_names, preds = multi_model_ensemble_test(models, weights, test_loader, device)
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
save_name = "soft_vote_3_4"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)
print(save_name, 'is saved!')

#### HardVote

1) 전체 출력물

    1-1) 가중치 없이 그냥
    
    1-2) 상위 3개의 만장일치에 우선순위
    
    1-3) 가중치 부여 (상위부터 0.3, 0.2, 0.1, 0.0, 0.0, ... ,0.0)
    
2) -0.005까지의 결과물들

    2-1) 가중치 없이 그냥
    
    2-2) 상위 3개의 만장일치에 우선순위

    2-3) 가중치 부여 (상위부터 0.3, 0.2, 0.1, 0.0, 0.0, ... ,0.0)

3) -0.01까지의 결과물들

    3-1) 가중치 없이 그냥
    
    3-2) 상위 3개의 만장일치에 우선순위
    
    3-3) 가중치 부여 (상위부터 0.3, 0.2, 0.1, 0.0, 0.0, ... ,0.0)

In [11]:
import random

def hard_vote_models (csv_files, best_model, isSecond = False, isHasWeights = False, weights = None) :
    out_submission = pd.DataFrame(columns=["image_id", "PredictionString"])
    out_submission["image_id"] = csv_files[0]["image_id"]
    #print(out_submission.head())
    
    whole_prediction = list()
    for i in tqdm(range(len(csv_files[0]))) :
        predict_arr = np.zeros(256*256)
        predict_arr_tmp = np.zeros((256*256, 12))
        
        for k, csv_file in enumerate(csv_files) :
            #print(csv_file.iloc[i])
            predict = csv_file.iloc[i]["PredictionString"]
            predict = predict.split()
            
            for j, pixel_predict in enumerate(predict) :
                if isHasWeights : 
                    predict_arr_tmp[j][eval(pixel_predict)] += 1 * weights[k]
                else :
                    predict_arr_tmp[j][eval(pixel_predict)] += 1
            
        for j, _ in enumerate(predict_arr_tmp) :
            max_index = np.argmax(predict_arr_tmp[j])
            max_index_size = np.where(predict_arr_tmp[j] == predict_arr_tmp[j][max_index])
            #print(max_index_size)
            if isSecond :
                if len(max_index_size) == 1 :
                    predict_arr[j] = max_index
                elif csv_files[0].iloc[i].split()[j] == csv_files[1].iloc[i].split()[j] and csv_files[0].iloc[i].split()[j] == csv_files[2].iloc[i].split()[j] :
                    predict_arr[j] = csv_files[0].iloc[i].split()[j]
                elif csv_files[best_model].iloc[i].split()[j] in max_index_size :
                    predict_arr[j] = csv_files[best_model].iloc[i].split()[j]
                else :
                    predict_arr[j] = random.choice(max_index_size)
            else :
                if len(max_index_size) == 1 :
                    predict_arr[j] = max_index
                elif csv_files[best_model].iloc[i].split()[j] in max_index_size :
                    predict_arr[j] = csv_files[best_model].iloc[i].split()[j]
                else :
                    predict_arr[j] = random.choice(max_index_size)
                
        whole_prediction.append(' '.join(str(int(e)) for e in predict_arr.tolist()))
    
    out_submission["PredictionString"] = whole_prediction
    return out_submission
        

In [ ]:

sub_dir = "./Camper_submissions/"
files = ["output.csv", \
         "soft_vote_3_2.csv", \
         "output (1).csv", \
         "soft_vote_2_1.csv", \
         "output (2).csv",\
         "soft_vote_2_2.csv", \
         "soft_vote_3_3.csv", \
         "soft_vote_3_4.csv", \
         "soft_vote_2_3.csv", \
         "soft_vote_2_4.csv", \
         "ensemble.csv", \
         "hardvote_ensemble2.csv", \
         "ensemble1.csv",\
         "hardvote_ensemble1.csv", \
         "output (3).csv", \
        ]

for i, file in enumerate(files) :
    files[i] = pd.read_csv(sub_dir + file)
#-----------------1-1
#파라미터 중 0 은 가장 좋은 모델의 index입니다. => 중복된 투표 수를 가진 항목들에 대해 가장 좋은 모델의 선택을 따를 예정.
submission = hard_vote_models(files, 0) 
save_name = "hard_vote_1_1"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)

#-----------------1-2
#파라미터 중 0 은 가장 좋은 모델의 index입니다. => 중복된 투표 수를 가진 항목들에 대해 가장 좋은 모델의 선택을 따를 예정.
submission = hard_vote_models(files, 0, isSecond = True) 
save_name = "hard_vote_1_2"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)

#-----------------1-3
weights = list()

for i, file in enumerate(files) :
    weight = 1 + 0.1*(3-i)
    if weight <= 1 : weight = 1
    weights.append(weight)
    
#파라미터 중 0 은 가장 좋은 모델의 index입니다. => 중복된 투표 수를 가진 항목들에 대해 가장 좋은 모델의 선택을 따를 예정.
submission = hard_vote_models(files, 0, isHasWeights = True, weights=weights) 
save_name = "hard_vote_1_3"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)

 10%|▉         | 80/837 [07:33<1:12:15,  5.73s/it]

In [ ]:
#-----------------2-1
sub_dir = "./Camper_submissions/"
files = ["output.csv", \
         "soft_vote_3_2.csv", \
         "output (1).csv"]

for i, file in enumerate(files) :
    files[i] = pd.read_csv(sub_dir + file)
    
#파라미터 중 0 은 가장 좋은 모델의 index입니다. => 중복된 투표 수를 가진 항목들에 대해 가장 좋은 모델의 선택을 따를 예정.
submission = hard_vote_models(files, 0) 
save_name = "hard_vote_2_1"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)

#-----------------2-2
#파라미터 중 0 은 가장 좋은 모델의 index입니다. => 중복된 투표 수를 가진 항목들에 대해 가장 좋은 모델의 선택을 따를 예정.
submission = hard_vote_models(files, 0, isSecond = True) 
save_name = "hard_vote_2_2"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)

#-----------------2-3
weights = list()

for i, file in enumerate(files) :
    weight = 1 + 0.1*(3-i)
    if weight <= 1 : weight = 1
    weights.append(weight)
    
#파라미터 중 0 은 가장 좋은 모델의 index입니다. => 중복된 투표 수를 가진 항목들에 대해 가장 좋은 모델의 선택을 따를 예정.
submission = hard_vote_models(files, 0, isHasWeights = True, weights=weights) 
save_name = "hard_vote_2_3"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)

In [ ]:
#-----------------3-1
sub_dir = "./Camper_submissions/"
files = ["output.csv", \
         "soft_vote_3_2.csv", \
         "output (1).csv", \
         "soft_vote_2_1.csv", \
         "output (2).csv",\
         "soft_vote_2_2.csv", \
         "soft_vote_3_3.csv", \
         "soft_vote_3_4.csv", \
         "soft_vote_2_3.csv", \
         "soft_vote_2_4.csv"]

for i, file in enumerate(files) :
    files[i] = pd.read_csv(sub_dir + file)
    
#파라미터 중 0 은 가장 좋은 모델의 index입니다. => 중복된 투표 수를 가진 항목들에 대해 가장 좋은 모델의 선택을 따를 예정.
submission = hard_vote_models(files, 0) 
save_name = "hard_vote_3_1"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)

#-----------------3-2
    
#파라미터 중 0 은 가장 좋은 모델의 index입니다. => 중복된 투표 수를 가진 항목들에 대해 가장 좋은 모델의 선택을 따를 예정.
submission = hard_vote_models(files, 0, isSecond = True) 
save_name = "hard_vote_3_2"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)

#-----------------3-3

weights = list()

for i, file in enumerate(files) :
    files[i] = pd.read_csv(sub_dir + file)
    weight = 1 + 0.1*(3-i)
    if weight <= 1 : weight = 1
    weights.append(weight)
    
#파라미터 중 0 은 가장 좋은 모델의 index입니다. => 중복된 투표 수를 가진 항목들에 대해 가장 좋은 모델의 선택을 따를 예정.
submission = hard_vote_models(files, 0, isHasWeights = True, weights=weights) 
save_name = "hard_vote_3_3"
submission.to_csv("./ensemble_result/"+save_name+".csv", index=False)

##### 결과 확인

In [ ]:
import pandas as pd
import numpy as np

index = 5 #이거 바꿔가면서 실행해서 너무 같은 값만 나오지 않는 지 확인해주세요!

sub = pd.read_csv('ensemble_result/hard_vote_1_1.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('hardvote_1_1')
print(arr)

sub = pd.read_csv('ensemble_result/hard_vote_1_2.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('hardvote_1_2')
print(arr)

sub = pd.read_csv('ensemble_result/hard_vote_1_3.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('hardvote_1_3')
print(arr)

#------------2번 하신 분은 여기만

sub = pd.read_csv('ensemble_result/hard_vote_2_1.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_2_1')
print(arr)

sub = pd.read_csv('ensemble_result/hard_vote_2_2.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('hardvote_2_2')
print(arr)

sub = pd.read_csv('ensemble_result/hard_vote_2_3.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('hardvote_2_3')
print(arr)

#--------------3번 하신 분은 여기만

sub = pd.read_csv('ensemble_result/hard_vote_3_1.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('softvote_3_1')
print(arr)

sub = pd.read_csv('ensemble_result/hard_vote_3_2.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('hardvote_3_2')
print(arr)

sub = pd.read_csv('ensemble_result/hard_vote_3_3.csv')
s = sub.iloc[index]["PredictionString"]

arr = np.zeros(12)
for ss in s.split() :
    arr[int(ss)] += 1
print('hardvote_3_3')
print(arr)
